import pandas as pd

In [1]:
import pandas as pd

In [2]:
tripadv_review = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Nicosia Mall.csv', error_bad_lines=False);

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [6]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0     [disable, person, person, need, services, disa...
1     [mall, okay, said, fairly, expensive, shops, f...
2     [expected, shopping, mall, cyprus, seen, kings...
3     [like, nicosia, mall, spacious, easy, park, he...
4     [nicosia, mall, beautiful, mall, lots, shops, ...
5     [decided, visit, nicosia, wish, hadn, wasted, ...
6     [went, today, enjoyed, experience, loads, shop...
7     [wish, spend, shopping, nicosia, mall, shops, ...
8     [mall, spacious, nice, shops, like, mainly, ci...
9     [large, shopping, center, stores, places, cine...
10    [nicosia, mall, looks, like, exact, paste, cop...
11    [biggest, mall, island, literally, hundreds, c...
12    [definitely, best, destination, cyprus, shoppi...
13    [nice, mall, choises, clothes, shoes, shop, li...
14    [standards, smallish, mall, good, range, cloth...
15    [nicosia, mall, planned, located, away, main, ...
16    [easy, parking, shops, choose, food, court, go...
17    [nice, restaurant, excellent, food, cosy, 

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 3), (18, 2), (19, 1), (20, 2), (21, 2), (22, 1)]]


In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=50,
                                           passes=3,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 10,
                                           eval_every = None)

In [9]:
from pprint import pprint

In [10]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.034*"shops" + 0.034*"nice" + 0.030*"mall" + 0.016*"cinema" + '
  '0.014*"parking" + 0.014*"like" + 0.014*"nicosia" + 0.013*"restaurants" + '
  '0.013*"cafe" + 0.012*"supermarket"'),
 (1,
  '0.025*"best" + 0.023*"shopping" + 0.019*"experience" + 0.017*"malls" + '
  '0.017*"cyprus" + 0.016*"excellent" + 0.016*"choice" + 0.014*"shops" + '
  '0.014*"worth" + 0.014*"great"'),
 (2,
  '0.024*"mall" + 0.018*"best" + 0.017*"food" + 0.014*"friendly" + '
  '0.014*"place" + 0.012*"island" + 0.011*"great" + 0.011*"shops" + '
  '0.011*"good" + 0.011*"shopping"'),
 (3,
  '0.042*"shops" + 0.041*"mall" + 0.020*"nicosia" + 0.017*"shopping" + '
  '0.014*"cyprus" + 0.013*"lots" + 0.013*"food" + 0.013*"places" + '
  '0.012*"court" + 0.012*"parking"'),
 (4,
  '0.029*"nicosia" + 0.026*"exit" + 0.019*"great" + 0.018*"shop" + '
  '0.017*"shops" + 0.012*"nice" + 0.010*"floor" + 0.010*"variety" + '
  '0.010*"close" + 0.010*"location"'),
 (5,
  '0.025*"great" + 0.020*"shops" + 0.018*"parking" + 0.015*"b

In [11]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.075914 -0.028836       1        1  29.721064
3      0.056636 -0.015591       2        1  21.366854
2     -0.100267 -0.048356       3        1  16.391525
5     -0.016601  0.098307       4        1  14.514356
0     -0.001488 -0.032889       5        1   9.747644
1     -0.013524  0.013613       6        1   4.471729
4     -0.000670  0.013751       7        1   3.786829, topic_info=         Term       Freq      Total Category  logprob  loglift
56    nicosia  10.000000  10.000000  Default  30.0000  30.0000
128      nice   6.000000   6.000000  Default  29.0000  29.0000
161     great   8.000000   8.000000  Default  28.0000  28.0000
172      best   6.000000   6.000000  Default  27.0000  27.0000
42   shopping   8.000000   8.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
11       mall   0.216866  21.915419   Topic7  -4.9799  -1.3420
94     cinema   0.130425   7.535421   Topic7  -5.4883  -0.7829
15    parking   0.131063   9.143831   Topic7  -5.4835  -0.9715
88      visit   0.099146   4.890198   Topic7  -5.7626  -0.6248
49     coffee   0.098464   5.362593   Topic7  -5.7695  -0.7239

[370 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
105       5  0.882470      adidas
221       1  0.449667       areas
171       4  0.510965  atmosphere
222       1  0.457110   available
3         1  0.660620        away
...     ...       ...         ...
91        2  0.562402        wish
48        2  0.570919       worth
48        6  0.285459       worth
119       1  0.520960        zara
119       5  0.520960        zara

[345 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 3, 6, 1, 2, 5])